In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
import requests
import time
import datetime as dt
import pandas as pd
from urllib.request import urlopen
import re
import datetime
import pymysql

In [2]:
"""
월초, 월말 리스트 저장
freq: ms-월초 m-월말
"""
today = datetime.date.today().strftime('%Y.%m.%d')
start_dates = pd.date_range(start = '2008-07-01', end = today, freq = 'MS')
end_dates = pd.date_range(start = '2008-07-01', end = today, freq = 'm')

def getDate(start_dates, end_dates, i):
    start_dates = start_dates.strftime('%Y.%m.%d')
    end_dates = end_dates.strftime('%Y.%m.%d')
    
    return start_dates[i], end_dates[i]

In [3]:
"""
검색 target: 네이버뉴스
검색 keyword: 
기간: start_date ~ end_date(한 달)
"""
def getNewsCountingFromHTML(start_date, end_date):
    raw_html = urlopen('https://search.naver.com/search.naver?where=news&query=%EB%B6%88%ED%99%A9&sm=tab_opt&sort=0&photo=0&field=0&reporter_article=&pd=3&ds='+start_date+'&de='+end_date+'&docid=&nso=so%3Ar%2Cp%3A')
    html = BeautifulSoup(raw_html, 'html.parser')
    count_tag = html.find_all("div", {"class": "title_desc all_my"})
    news_cnt = count_tag[0].get_text()[7:]
    
    return news_cnt

In [4]:
"""
'keyword에 대한 뉴스 기사 건수
str을 int로 변환 
ex. 3,277건 --> 3277
"""
def extract_only_counting(text):
    regex = re.compile(r'((-)?\d{1,3}(,\d{3})*(\.\d+)?)')
    search = regex.search(text)
    
    if search:
        return int(search.group(1).replace(',',''))
    else:
        return 0

In [5]:
# for i in range(len(start_dates)):
    
#     start_date, end_date = getDate(start_dates, end_dates, i)
#     print(start_date, end_date)
#     news_cnt = getNewsCountingFromHTML(start_date, end_date)
#     news_cnt = extract_only_counting(news_cnt)
#     print(news_cnt)
#     print() 
#     time.sleep(1.3)
# ## db에 저장만 하면될듯.

### DB연동

In [18]:
conn = pymysql.connect(host = "127.0.0.1", user = [USER], passwd = [PASSWORD], db = [DATABASE])
cur = conn.cursor()

In [7]:
# cur.execute("show databases")
# print(cur.fetchall())
# cur.execute("use CLI")

(('information_schema',), ('CLI',), ('MusicBank',), ('mysql',), ('mytestdb',), ('performance_schema',), ('sys',))


0

### DB table 생성

In [8]:
"""
news counting 정보 저장할 table 생성
"""

create_table_query = """
CREATE TABLE IF NOT EXISTS news_counting(
    id BIGINT(7) NOT NULL AUTO_INCREMENT,
    year bigint(4) NOT NULL,
    month bigint(2) NOT NULL,
    cnt bigint(100),
    primary key(id) )
    charset=utf8mb4;
"""
cur.execute(create_table_query)
# charset=utf8mb4; 유니코드 스트링 저장

0

In [19]:
cur.execute("desc news_counting")
table = cur.fetchall()
pd.DataFrame(list(table))

,0,1,2,3,4,5
0,id,bigint(7),NO,PRI,None,auto_increment
1,year,bigint(4),NO,,None,
2,month,bigint(2),NO,,None,
3,cnt,bigint(100),YES,,None,


### news counting정보 달별로 저장

In [15]:
"""
db table에 저장
"""

for i in range(len(start_dates)-1):
    query = """ 
    Insert into news_counting (year,month, cnt) values (%d, %d, %d) ;
    """
    start_date, end_date = getDate(start_dates, end_dates, i)
    news_cnt = getNewsCountingFromHTML(start_date, end_date)
    news_cnt = extract_only_counting(news_cnt)

    mystring = (query % (int(start_date[0:4]), int(start_date[5:7]), news_cnt))
    print(mystring)
    cur.execute(mystring)

 
    Insert into news_counting (year,month, cnt) values (2008, 7, 1696) ;
    
 
    Insert into news_counting (year,month, cnt) values (2008, 8, 1544) ;
    
 
    Insert into news_counting (year,month, cnt) values (2008, 9, 1742) ;
    
 
    Insert into news_counting (year,month, cnt) values (2008, 10, 3808) ;
    
 
    Insert into news_counting (year,month, cnt) values (2008, 11, 7020) ;
    
 
    Insert into news_counting (year,month, cnt) values (2008, 12, 9974) ;
    
 
    Insert into news_counting (year,month, cnt) values (2009, 1, 9234) ;
    
 
    Insert into news_counting (year,month, cnt) values (2009, 2, 8297) ;
    
 
    Insert into news_counting (year,month, cnt) values (2009, 3, 9692) ;
    
 
    Insert into news_counting (year,month, cnt) values (2009, 4, 7451) ;
    
 
    Insert into news_counting (year,month, cnt) values (2009, 5, 5400) ;
    
 
    Insert into news_counting (year,month, cnt) values (2009, 6, 4567) ;
    
 
    Insert into news_counting (year

In [16]:
# 확인
query = """ 
select * from news_counting ;
"""
cur.execute(query)
pd.DataFrame(list(cur.fetchall()))

,0,1,2,3
0,1,2008,7,1696
1,2,2008,8,1544
2,3,2008,9,1742
3,4,2008,10,3808
4,5,2008,11,7020
...,...,...,...,...
142,143,2020,5,4294
143,144,2020,6,4328
144,145,2020,7,3743
145,146,2020,8,3278


In [17]:
conn.commit()
cur.close()
conn.close()